# Contagion Window Model

See this GitHub repo for the full version: https://github.com/samkennerly/covid19

## abstract

A "contagion window" is a fixed period of time during which each infected host can transmit an infection to susceptible hosts. In this model, the expected transmission count per day is adjustable for each day in the window. Each window coefficient plays the same role as $\beta$ in a [SIR model]. Unlike SIR, the master equation of this model is a discrete difference equation which uses [convolution] to capture time-sensitive details.

In this model, infected persons are assumed to quarantine themselves when symptoms appear. For an infection whose symptoms appear several days *after* a person becomes infectious, it may already be too late to "close the window."

The parameters in this notebook are examples meant to be replaced by expert epidemiologists. (The author is not one.)
 
[SIR model]: https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model
[convolution]: https://en.wikipedia.org/wiki/Convolution

In [ ]:
from datetime import timedelta
from numpy import linspace
from pathlib import Path
from pandas import DataFrame, Series, read_csv

FIGSIZE = (9, 5)
FOLDER = Path("../input/covid19-local-us-ca-forecasting-week-1")

## get data
Load training data and prediction dates.

In [ ]:
train = ( 
    read_csv(FOLDER / "ca_train.csv", parse_dates=['Date'])
    ['Id Date ConfirmedCases Fatalities'.split()]
    .set_index('Date')
)
train.tail(20)

In [ ]:
forecast_id = read_csv(FOLDER / "ca_test.csv", parse_dates=['Date']).set_index('Date')['ForecastId']
forecast_id.tail()

## model

In this example, infected persons become immediately contagious, then quarantine themselves a few days later when symptoms appear.

In [ ]:
def contagion_window(delay=4, duration=14, reproduction=2.5):
    window = Series(linspace(1, 0, duration), name='window')
    window *= reproduction / window.sum()
    window[delay:] = 0
    
    return window.rename_axis('days since infection')

axes = contagion_window().plot.bar(color='red', figsize=FIGSIZE, title='expected transmissions')

In [ ]:
def convolved(values, window):
    """ int or float: Discrete convolution of two timeseries. """
    return sum( w * x for w, x in zip(window, reversed(values)) )

def predict(train, dates, confirm_rate=0.2, population=40_000_000, vax_rate=0, **kwargs):
    """ DataFrame: Predicted future cases and fatalities. """

    aday = timedelta(days=1)
    data = DataFrame(index=dates)
    window = contagion_window(**kwargs)
    duration = len(window)
    new_days = data.index.difference(train.index)

    # Backfill data from training-testing overlap
    data['confirmed'] = train['ConfirmedCases']
    data['deceased'] = train['Fatalities']
    
    # Estimate infection counts and real mortality rate
    data['confirmed'] = train['ConfirmedCases']
    data['exposed'] = (train['ConfirmedCases'] / confirm_rate)
    mortality = data['deceased'].sum() / data['exposed'].sum()
    print("Estimated mortality:", round(100 * mortality, 2), "%")
 
    # Predict new cases
    data['new'] = data['exposed'].diff()
    data.at[dates.min(), 'new'] = data['exposed'][0]
    for t in new_days:
        deltas = data.loc[:t-aday, 'new']
        data.at[t, 'new'] = (1 - vax_rate - sum(deltas) / population ) * convolved(deltas, window)
    
    # Calculate cumulative totals
    data.loc[new_days, 'exposed'] = data['new'].cumsum()
    data.loc[new_days, 'deceased'] = data.loc[new_days, 'exposed'] * mortality
    data.loc[new_days, 'confirmed'] = data.loc[new_days, 'exposed'] * confirm_rate

    return data.sort_index(axis=1).astype(int)
    
data = predict(train, forecast_id.index)
axes = data.plot.line(color='bkgr', figsize=FIGSIZE, logy=True)
data.tail()

## submit results

In [ ]:
submission = DataFrame(index=data.index)
submission['ForecastId'] = forecast_id
submission['ConfirmedCases'] = data['confirmed']
submission['Fatalities'] = data['deceased']
submission.to_csv('submission.csv', index=False)